In [6]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import simpy
from math import ceil

In [75]:
# Supplier
attach_rate = 0.5
supplier_capacity = 1000
supply_unit = 40
supply_deliver_unit = 32
supply_logistic_lead_time = 2

# Manufacturer
manufacturer_capacity = 1000
manufacturing_capacity = 10
manufacture_deliver_unit = 8
manufacture_logistic_lead_time = 2

SimPy is (like most simulation frameworks) a single-threaded, deterministic library. It processes events sequentially – one after another. If two events are scheduled at the same time, the one that is scheduled first will also be the processed first (FIFO) </br>

Container vs. Resource vs. Store
* Container : Containers model the production and consumption of a homogeneous, undifferentiated bulk. It may either be continuous (like water) or discrete (like apples).
* Resource : These resources can be used by a limited number of processes at a time (e.g., a gas station with a limited number of fuel pumps)
* Store : Stores model the production and consumption of concrete objects. The object type is, by default, not restricted. A single Store can even contain multiple types of objects.

In [113]:
class Supplier(object):
    def __init__(self, env, manufacturefacility, supplier_capacity, supply_unit, deliver_unit, logistic_lead_time):
        self.env = env

        # Container - can fill & consume 1 raw material
        self.capacity = simpy.Container(env, capacity = supplier_capacity, init = 0)

        # Async Processes
        self.env.process(self.supply(supply_unit))
        self.env.process(self.deliver(deliver_unit, logistic_lead_time, manufacturefacility))

    def supply(self, supply_unit):
        while True:
            # Collect raw materials (supply_unit) daily
            yield self.env.timeout(1)
            yield self.capacity.put(supply_unit)
            print("Current supplier inventory is %d at %d" % (self.capacity.level, self.env.now))
            
    def deliver(self, deliver_unit, logistic_lead_time, manufacturefacility):
        while True:
            # If there are enough units, deliver after lead time
            if deliver_unit <= self.capacity.level:
                yield self.env.timeout(logistic_lead_time)
                # Get units from supplier then put units to manufacturer
                event_get =  self.capacity.get(deliver_unit)
                event_put =  manufacturefacility.raw_material.put(deliver_unit)
                yield self.env.all_of([event_get, event_put])
                print("Deliver %d units to Manufacturer at %d" % (deliver_unit, self.env.now))
            # If not, try delivery next day
            else:
                yield self.env.timeout(1)


class ManufactureFacility(object):
    def __init__(self, env, manufacturing_capacity, attach_rate, manufacturer_capacity, deliver_unit, logistic_lead_time) :
        # super().__init__(sim=sim)
        self.env = env
        self.manufacturing_capacity = manufacturing_capacity
        self.manufacturer_capacity = manufacturer_capacity

        # Container for raw material & manufactured goods
        self.raw_material = simpy.Container(env, capacity=(manufacturer_capacity/attach_rate), init=0)
        self.capacity = simpy.Container(env, capacity=manufacturer_capacity, init=0)

        # Async Processes
        self.env.process(self.manufacture(attach_rate))
        self.env.process(self.deliver(deliver_unit, logistic_lead_time))
        # yield env.any_of([event_a, event_b, event_c])

    def check_supplier_delivery(self):
        while True:
            yield self.env.timeout(1)
            check = self.raw_material.level
            return check

    def manufacture(self, attach_rate):
        while True:
            # Manufacture using all existing raw materials but within manufacturing capacity
            units = round(attach_rate * self.raw_material.level)
            units = min(units, self.manufacturing_capacity)

            # Manufacture goods daily
            if units > 0:
                # manufacture goods immediately
                yield self.capacity.put(units)
                yield self.raw_material.get(units/attach_rate)
                print("Manufacture %d units at %d" % (units, self.env.now))
                print("Current manufactured goods inventory is %d at %d" % (self.capacity.level, self.env.now))
                # Start the new day
                yield self.env.timeout(1)
            else:
                # No action required, wait until there's enough raw materials
                yield self.env.timeout(1)    


    def deliver(self, deliver_unit, logistic_lead_time):
        while True:
            # If there are enough units, deliver after lead time
            if deliver_unit <= self.capacity.level:
                yield self.env.timeout(logistic_lead_time)
                yield self.capacity.get(deliver_unit)
                print("Deliver %d units to Distribution Hub at %d" % (deliver_unit, self.env.now))
            # If not, try delivery next day
            else:
                yield self.env.timeout(1)



In [114]:
env = simpy.Environment()
manufacturer = ManufactureFacility(env,
      manufacturer_capacity=manufacturer_capacity,
      attach_rate = attach_rate,
      manufacturing_capacity=manufacturing_capacity,
      deliver_unit=manufacture_deliver_unit,
      logistic_lead_time=manufacture_logistic_lead_time)

supplier = Supplier(env, manufacturefacility=manufacturer,\
      supplier_capacity=supplier_capacity,
      supply_unit = supply_unit,
      deliver_unit=supply_deliver_unit,
      logistic_lead_time=supply_logistic_lead_time)

env.run(until=12)

Current supplier inventory is 40 at 1
Current supplier inventory is 80 at 2
Current supplier inventory is 88 at 3
Deliver 32 units to Manufacturer at 3
Manufacture 10 units at 3
Current manufactured goods inventory is 10 at 3
Current supplier inventory is 128 at 4
Manufacture 6 units at 4
Current manufactured goods inventory is 16 at 4
Deliver 8 units to Distribution Hub at 5
Current supplier inventory is 136 at 5
Deliver 32 units to Manufacturer at 5
Manufacture 10 units at 5
Current manufactured goods inventory is 18 at 5
Current supplier inventory is 176 at 6
Manufacture 6 units at 6
Current manufactured goods inventory is 24 at 6
Deliver 8 units to Distribution Hub at 7
Current supplier inventory is 184 at 7
Deliver 32 units to Manufacturer at 7
Manufacture 10 units at 7
Current manufactured goods inventory is 26 at 7
Current supplier inventory is 224 at 8
Manufacture 6 units at 8
Current manufactured goods inventory is 32 at 8
Deliver 8 units to Distribution Hub at 9
Current suppl

In [60]:
supplier.capacity.level

60